In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

In [2]:
def process_csv(folder_path):
    input_csv_file_path = os.path.join(folder_path, 'processed_step_3.csv')
    output_csv_file_path = os.path.join(folder_path, 'processed_step_4.csv')

    df = pd.read_csv(input_csv_file_path)

    origins = df[['origin_x', 'origin_y']].values
    goals = df[['goal_x', 'goal_y']].values

    origin_z = []
    goal_z = []

    for i in range(len(df)):
        depth_path = os.path.join(folder_path, 'processed_depth', f'depth_data_{i + 1}.bin')
        if not os.path.exists(depth_path):
            origin_z.append(None)
            goal_z.append(None)
            continue
        
        origin = origins[i]
        goal = goals[i]
        
        with open(depth_path, 'rb') as f:
            depth_data = f.read()
            depth_array = np.frombuffer(depth_data, dtype=np.uint16)
            width = 640
            height = 480
            depth_frame = depth_array.reshape((height, width)) / 10000

            padding_x = -10
            padding_y = -10
            
            # clear_output(wait=True)
            # plt.figure(figsize=(5, 5))
            # plt.imshow(depth_frame, cmap='viridis')
            # if not np.isnan(origin).any():
            #     plt.scatter(origin[0] + padding_x, origin[1] + padding_y, c='red', marker='x', label='Origin')
            # if not np.isnan(goal).any():
            #     plt.scatter(goal[0] + padding_x, goal[1] + padding_y, c='blue', marker='x', label='Goal')
            # plt.title(depth_path)
            # display(plt.gcf())
        
            origin_depth = depth_frame[int(origin[1] + padding_y), int(origin[0] + padding_x)] if not np.isnan(origin).any() else None
            goal_depth = depth_frame[int(goal[1] + padding_y), int(goal[0] + padding_x)] if not np.isnan(goal).any() else None
            
            origin_z.append(origin_depth)
            goal_z.append(goal_depth)

    df['origin_z'] = origin_z
    df['goal_z'] = goal_z

    df.to_csv(output_csv_file_path, index=False)
    plt.clf()

In [3]:
base_folder = r'.\NaoPickAndPlaceData'
for folder in os.listdir(base_folder):
    folder_path = os.path.join(base_folder, folder)
    if os.path.isdir(folder_path) and 'processed_step_3.csv' in os.listdir(folder_path) and 'processed_step_4.csv' not in os.listdir(folder_path):
        process_csv(folder_path)

<Figure size 640x480 with 0 Axes>